# Class Exercises - Week 2

---
---

**Author:** Dr Giordano Scarciotti (g.scarciotti@imperial.ac.uk) - Imperial College London 

**Module:** ELEC70066 - Advanced Optimisation

**Version:** 1.1.1 - 09/01/2023

---
---

**Reminder:** The solutions of some these exercises may be found online. Do not look for the solutions, but ask for help to me or the GTA. Your assessment will not have a solution available online so it is of paramount importance that you learn how to solve the problems without relying on a solution being available. 

The solutions of these exercises will be discussed at the end of the session. Moreover, the complete solution will be provided to you after 1 week. **N.B.** **Do not share the solutions online**

In [ ]:
import cvxpy as cp
import numpy as np

## Exercise 1

Consider the problem 

$$
\begin{array}{ll}
\displaystyle \min_{x,y} & f_0(x,y)\\ 
\text{s.t. } & 2 x + y \ge 1\\
& x + 3 y \ge 1\\
& x \ge 0\\
& y \ge 0
\end{array}
$$

Write a programme in python that solves the four problems for

1.   $f_0(x, y) = x + y$
2.   $f_0(x, y) = -x - y$
3.   $f_0(x, y) = \max(x,y)$
4.   $f_0(x, y) = x^2 + 9y^2$


Comment all your code and print the optimal $x$, $y$ and value of the problem for the four cases above.

In [ ]:
x = cp.Variable()
y = cp.Variable()

constraints = [2*x + y>= 1, 
               x + 3*y >= 1,
               x >= 0,
               y >= 0]


In [ ]:
#1
objective = cp.Minimize(x+y)
prob = cp.Problem(objective,constraints)
prob.solve()
print("status:", prob.status)
print("Optimal value", prob.value)
print("Optimal var of x is {} , y is {}".format(x.value,y.value))

status: optimal
Optimal value 0.5999999999116253
Optimal var of x is 0.3999999999724491 , y is 0.1999999999391762


In [ ]:
#2
objective = cp.Minimize(-x-y)
prob = cp.Problem(objective,constraints)
prob.solve()
print(prob.status)
# this is probably negative infinity

unbounded


In [ ]:
#3
objective = cp.Minimize(cp.maximum(x,y))
prob = cp.Problem(objective,constraints)
prob.solve()
print("status:", prob.status)
print("Optimal value", prob.value)
print("Optimal var of x is {} , y is {}".format(x.value,y.value))

status: optimal
Optimal value 0.3333333334080862
Optimal var of x is 0.3333333334080862 , y is 0.33333333286259564


In [ ]:
#4
objective = cp.Minimize(x**2 + 9 * y**2)
prob = cp.Problem(objective,constraints)
prob.solve()
print("status:", prob.status)
print("Optimal value", prob.value)
print("Optimal var of x is {} , y is {}".format(x.value,y.value))

status: optimal
Optimal value 0.5000000000000002
Optimal var of x is 0.5000000000000001 , y is 0.1666666666666667


# Exercise 2


This exercise is adapted from the example "Fitting censored data" on the CVXPY website. **Do not look for the solution**, but ask for help to me or the GTA. You can use any other example on the CVXPY website, but not the "Fitting censored data" one. Your assessment will not have a solution available online so it is of paramount importance that you learn how to solve the problems without relying on a solution being available. 

In statistics, [censoring](https://en.wikipedia.org/wiki/Censoring_(statistics)) is a condition in which the value of a measurement or observation is only partially known. 

For instance, suppose a study is conducted to measure the impact of a drug on mortality rate. In such a study, it may be known that an individual's age at death is at least 75 years (but may be more). Such a situation could occur if the individual withdrew from the study at age 75, or if the individual is currently alive at the age of 75.

Censoring also occurs when a value occurs outside the range of a measuring instrument. For example, a bathroom scale might only measure up to 140 kg. If a 160-kg individual is weighed using the scale, the observer would only know that the individual's weight is at least 140 kg. 



We wish to fit a set of data points,

$$
(x^{(1)},y^{(1)}), \dots , (x^{(K)},y^{(K)}),
$$

with $x^{(k)}\in \mathbb{R}^n$ and $y^{(k)} \in \mathbb{R}$, with a linear model of the form $y \approx c^\top x$. The vector $c\in\mathbb{R}^n$ is the model parameter, which we want to choose. We will use a least-squares criterion, i.e., choose $c$ to minimize

$$
J = \sum_{k=1}^K \left( y^{(k)} - c^\top x^{(k)} \right)^2
$$

All the values of $x^{(k)}$ are known. The data $y^{(1)}$, ..., $y^{(M)}$ are known, while the data $y^{(M+1)}$, ..., $y^{(K)}$ are all censored, i.e. unknown but larger than $D$, a known number.

Solve three problems:



1.   Solve the naive Least-Square problem (i.e. minimize $J$) considering the entire data from $1$ to $K$ as if it were correct (i.e. assume that the censored data is the actual data)
2.   Solve the Least-Squares problem ignoring the censored data, i.e. use only the data from $1$ to $M$
3.   Use constraints to take into account the censored data

For each of the problems compute the residual

$$
\frac{||c_{true} - \hat c||_2}{||c_{true}||_2}
$$

where $c_{true}$ is the actual value of $c$ and $\hat c$ is the estimated value in each of the three problems.

The data is generated by this snippet of code

In [ ]:
# This code generates the data for the problem
# The variables that you can use are X_ordered, y_censored and D.
# You can use c_true to compute the residual
# DO NOT USE X, y and y_ordered, these are internal variables
# used to generate the data for the problem

import cvxpy as cp
import numpy as np
n = 30 # number of variables
M = 50 # number of uncensored observations
K = 200 # total number of observations

# Random generation of the data
np.random.seed(n*M*K)
X = np.random.randn(K*n).reshape(K, n) # DO NOT USE
c_true = np.random.rand(n)
y = X.dot(c_true) + .3*np.sqrt(n)*np.random.randn(K) # DO NOT USE

# Ordering them based on y
order = np.argsort(y) # DO NOT USE
y_ordered = y[order] # DO NOT USE
X_ordered = X[order,:]

# finding boundary
D = (y_ordered[M-1] + y_ordered[M])/2.

# applying censoring
y_censored = np.concatenate((y_ordered[:M], np.ones(K-M)*D))

# The variables that you can use are X_ordered, y_censored and D.
# You can use c_true to compute the residual
# DO NOT USE X, y and y_ordered, these are internal variables
# used to generate the data for the problem

In [ ]:
# can use c_true, X_ordered

# 1- Ignore existence of censoring
c = cp.Variable(n)

#objective = cp.Minimize(cp.sum_squares(y- c.T @ X_ordered))

objective = cp.Minimize(cp.norm2(y_censored -  X_ordered@ c))

prob = cp.Problem(objective)

prob.solve()
print("status:", prob.status)
print("Optimal value", prob.value)
print("Optimal var c :", c.value, c.shape)

#residual = np.sum(np.square(c_true - c.value))/np.sum(np.square(c_true))

residual = np.linalg.norm(c_true-c.value)/np.linalg.norm(c_true)
c_value_1 = c.value
print("Residual is", residual)

status: optimal
Optimal value 41.21798477210765
Optimal var c : [ 0.21675412 -0.01055401  0.6084366   0.38871172 -0.02505805 -0.32893498
 -0.04523541 -0.13816865  0.48735647 -0.00860644  0.22483505  0.1980222
  0.23656382  0.22497391  0.06820719  0.10749049 -0.1314469   0.00584045
  0.04698078 -0.08254453  0.30340207  0.09028037 -0.20223001  0.18510921
  0.49060366  0.18815787  0.05200329  0.3670569  -0.034346    0.1419044 ] (30,)
Residual is 0.838156314402116


In [ ]:
#2 Use data up to index M
c = cp.Variable(n)
objective = cp.Minimize(cp.norm2(y_censored[:M] -  X_ordered[:M]@ c))

prob = cp.Problem(objective)

prob.solve()
print("status:", prob.status)
print("Optimal value", prob.value)
print("Optimal var c :", c.value, c.shape)

#residual = np.sum(np.square(c_true - c.value))/np.sum(np.square(c_true))

residual = np.linalg.norm(c_true-c.value)/np.linalg.norm(c_true)

c_value_2 = c.value

print("Residual is", residual)

status: optimal
Optimal value 5.804982935751006
Optimal var c : [ 0.72956219  0.59273637  1.49803854  1.26515038  0.90959927  0.27911708
  0.33039085  0.58718121  0.10692683  0.69727413  1.21295263  0.92209524
  0.62076821  0.74019248  0.01724733  0.758353   -0.3686797   1.45235794
  0.00563295  1.35378805  0.78189158  0.34821785 -0.18566106  0.4954472
  0.91181214  0.79014362  1.25547097  0.51207478  1.21229208  0.40918099] (30,)
Residual is 0.6752336824029347


Problem 3

In [ ]:
objective = cp.Minimize(cp.norm2(y_censored[:M] -  X_ordered[:M]@ c))
constraints = [ X_ordered[M:]@c >= D]

prob = cp.Problem(objective,constraints)

prob.solve()
print("status:", prob.status)
print("Optimal value", prob.value)
print("Optimal var c :", c.value, c.shape)

residual = np.sum(np.square(c_true - c.value))/np.sum(np.square(c_true))

residual = np.linalg.norm(c_true-c.value)/np.linalg.norm(c_true)

c_value_3 = c.value

print("Residual is", residual)

status: optimal
Optimal value 10.019202219262665
Optimal var c : [ 0.6197059   0.16289534  0.95608615  0.52516372  0.57954434  0.67022677
  0.088551    0.60130836 -0.27252818 -0.02518875  0.69818258  0.88241527
  0.43527415  0.88481418  0.11263475  0.60701193  0.10914102  1.07818788
  0.12440878  1.31124691  0.55172325 -0.15040255 -0.21464237  0.42826683
  0.48603722  0.53978605  0.85544729  1.20637473  0.94039366  0.72184268] (30,)
Residual is 0.4495982372873162


0.41664139868970756

# Exercise 3

What is the solution of the norm approximation problem with one **scalar** variable $x \in \mathbb{R}$

$$
\displaystyle\min_x || x \mathbf{1} - b ||
$$

where $b \in \mathbb{R}^m$, and $\mathbf{1}$ is a vector of all $1$'s, for the following cases:

1.   $\ell_2$-norm
2.   $\ell_1$-norm
3.   $\ell_\infty$-norm

You can use CVX on an example to try to guess the solution. Can you obtain the analytical formula for all three cases?